In [1]:
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import optuna

In [2]:
# local
root_path = '../data/open/머신러닝'

# colab
# root_path = '/content/drive/MyDrive/12조 파이널프로젝트/data'

In [3]:
# 데이터 불러오기
#train_file = '201812_vif_drop_All'
#train_file = '201812_corr_drop_All'
train_file = 'vif_one_segment'
train_df = pd.read_parquet(f'{root_path}/{train_file}.parquet')

test_file = f'{root_path}/test_12월_통합.parquet'
test_df = pd.read_parquet(test_file)

train_df

,ID,기준년월,소지카드수_유효_신용,소지카드수_이용가능_신용,입회경과개월수_신용,이용금액_R3M_신용_가족,이용여부_3M_해외겸용_본인,이용여부_3M_해외겸용_신용_본인,2순위신용체크구분_인코딩,CA한도금액,...,_3순위교통업종_이용금액,_1순위여유업종_이용금액,_1순위납부업종_이용금액,RP건수_B0M,RP유형건수_B0M,이용개월수_온라인_R6M,연속유실적개월수_기본_24M_카드,청구서발송여부_B0,잔액_일시불_B2M,Segment
0,TRAIN_000000,201807,1,1,67,0,0,0,0,7270,...,0,0,1883,2,2,1,13,1,1083,D
1,TRAIN_000000,201808,1,1,68,0,0,0,0,6293,...,0,0,1959,2,2,1,13,1,736,D
2,TRAIN_000000,201809,1,1,69,0,0,0,0,6449,...,0,0,2008,2,2,1,17,1,997,D
3,TRAIN_000000,201810,1,1,70,0,0,0,0,5990,...,0,0,1972,2,2,1,17,1,697,D
4,TRAIN_000000,201811,1,1,71,0,0,0,0,5816,...,0,0,1982,2,2,0,17,1,607,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,TRAIN_399999,201808,1,1,4,0,0,0,1,25757,...,0,0,0,0,0,0,2,1,0,E
2399996,TRAIN_399999,201809,1,1,5,0,0,0,1,26818,...,0,0,0,0,0,0,5,1,998,E
2399997,TRAIN_399999,201810,1,1,6,0,0,0,1,27543,...,0,0,0,0,0,0,5,1,814,E
2399998,TRAIN_399999,201811,1,1,7,0,0,0,1,27113,...,0,0,0,0,0,0,5,1,1485,E


### 1단계 -> Segment == E vs Other

In [5]:
train_df["Segment1"] = train_df["Segment"].apply(lambda x: "E" if x == "E" else "other")

In [6]:
exclude_cols = ['ID', '기준년월', 'Segment', 'Segment1']
feature_cols = [
    col for col in train_df.columns
    if col not in exclude_cols
]

In [7]:
# 학습용 피처/타겟 정의
X = train_df[feature_cols]
y = train_df['Segment1']

In [8]:
# X와 test_df 양쪽에 모두 존재하는 컬럼만 선택
common_cols = [col for col in X.columns if col in test_df.columns]
X = X[common_cols]
X_test = test_df[common_cols]

In [9]:
# 학습/검증 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [10]:
def objective(trial):
    params = {
        # boosting 반복 횟수
        "iterations": trial.suggest_int("iterations", 300, 1500),
        # 한번에 얼마나 빠르게 학습하지지
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
        # 트리의 깊이, 깊을수록 복잡한 모델이지만 과적합 가능성 O
        "depth": trial.suggest_int("depth", 4, 10), # --> 튜닝
        # L2 정규화 , 높을수록 규제 강화 - 과적합 억제
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 10),
        # 수치형 피쳐를 이산화 할 때 사용하는 경계 개수 -> 수치형 많으면 영향력 있음
        "border_count": trial.suggest_int("border_count", 32, 255),
        # 트리 분기시 f.i 계산에 노이즈를 얼마나 줄 것인가
        "random_strength": trial.suggest_float("random_strength", 0.1, 1),
        # 샘플링의 다양성 조절 -> 과소적합일 땐 높이고, 과적합일 땐 낮게
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0, 1),
        # 손실 함수 : 다중분류
        "loss_function": "Logloss",
        # 평가 지표
        "eval_metric": "F1",
        # 학습 로그를 몇 step마다 보여줄지
        "verbose": 100,
        # 트리 분기 시 feature 중요도 계산에 주는 노이즈 크기
        "random_state": 42
    }
    # early_stopping_rounds - 지정된 라운드 동안 성능 개선이 되지 않으면 학습 중지
    model = CatBoostClassifier(**params)
    model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50, verbose=100)

    preds = model.predict(X_val)
    f1 = f1_score(y_val, preds, average='micro')
    return f1

In [ ]:
# Optuna 튜닝 시작
study = optuna.create_study(direction="maximize")
# XGB 때 n_trials=30 으로 했었는데 너무 오래걸려서 일단 20으로 설정
# n_trials는 optuna가 시도할 하이퍼파라미터 조합의 개수 (몇번 학습할건지)
study.optimize(objective, n_trials=30)

[I 2025-07-14 15:28:44,519] A new study created in memory with name: no-name-bc5fb7db-3054-49bd-b893-6131e3d25f5f


0:	learn: 0.6618653	test: 0.6596647	best: 0.6596647 (0)	total: 623ms	remaining: 6m 47s
100:	learn: 0.7028027	test: 0.6996149	best: 0.6996149 (100)	total: 40.4s	remaining: 3m 41s
200:	learn: 0.7110169	test: 0.7072485	best: 0.7072485 (200)	total: 1m 20s	remaining: 3m 1s
300:	learn: 0.7157627	test: 0.7112245	best: 0.7112245 (300)	total: 1m 59s	remaining: 2m 20s
400:	learn: 0.7200644	test: 0.7140997	best: 0.7141622 (398)	total: 2m 40s	remaining: 1m 41s
500:	learn: 0.7237175	test: 0.7168858	best: 0.7168916 (499)	total: 3m 20s	remaining: 1m 1s
600:	learn: 0.7270859	test: 0.7192302	best: 0.7192302 (600)	total: 4m 1s	remaining: 21.7s
654:	learn: 0.7288778	test: 0.7203919	best: 0.7203919 (654)	total: 4m 22s	remaining: 0us

bestTest = 0.7203918998
bestIteration = 654



[I 2025-07-14 15:33:25,644] Trial 0 finished with value: 0.8986875 and parameters: {'iterations': 655, 'learning_rate': 0.05989790431031373, 'depth': 8, 'l2_leaf_reg': 8.447911809305529, 'border_count': 197, 'random_strength': 0.8981347546296288, 'bagging_temperature': 0.170963903717959}. Best is trial 0 with value: 0.8986875.


0:	learn: 0.6469787	test: 0.6451828	best: 0.6451828 (0)	total: 350ms	remaining: 6m 30s
100:	learn: 0.7027971	test: 0.7000728	best: 0.7000728 (100)	total: 33.6s	remaining: 5m 37s
200:	learn: 0.7094913	test: 0.7063537	best: 0.7063537 (200)	total: 1m 6s	remaining: 5m
300:	learn: 0.7131177	test: 0.7098196	best: 0.7098196 (300)	total: 1m 38s	remaining: 4m 26s
400:	learn: 0.7156864	test: 0.7116901	best: 0.7117315 (396)	total: 2m 11s	remaining: 3m 54s
500:	learn: 0.7182854	test: 0.7135494	best: 0.7136177 (499)	total: 2m 44s	remaining: 3m 21s
600:	learn: 0.7204277	test: 0.7148753	best: 0.7148762 (597)	total: 3m 17s	remaining: 2m 48s
700:	learn: 0.7220802	test: 0.7160989	best: 0.7161783 (691)	total: 3m 50s	remaining: 2m 15s
800:	learn: 0.7238668	test: 0.7178642	best: 0.7178666 (799)	total: 4m 22s	remaining: 1m 43s
900:	learn: 0.7254176	test: 0.7191820	best: 0.7191820 (900)	total: 4m 55s	remaining: 1m 10s
1000:	learn: 0.7271912	test: 0.7199206	best: 0.7200184 (993)	total: 5m 28s	remaining: 37.4s

[I 2025-07-14 15:39:49,793] Trial 1 finished with value: 0.89868125 and parameters: {'iterations': 1115, 'learning_rate': 0.08657051577426762, 'depth': 6, 'l2_leaf_reg': 7.050956470299588, 'border_count': 213, 'random_strength': 0.22983573426023, 'bagging_temperature': 0.6837681451042293}. Best is trial 0 with value: 0.8986875.


...

0:	learn: 0.6631853	test: 0.6620144	best: 0.6620144 (0)	total: 887ms	remaining: 13m 15s
100:	learn: 0.7027707	test: 0.6988399	best: 0.6988399 (100)	total: 1m 29s	remaining: 11m 49s
200:	learn: 0.7141197	test: 0.7083607	best: 0.7083607 (200)	total: 2m 55s	remaining: 10m 10s
300:	learn: 0.7208799	test: 0.7140629	best: 0.7140629 (300)	total: 4m 21s	remaining: 8m 38s
400:	learn: 0.7267138	test: 0.7181069	best: 0.7181323 (399)	total: 5m 50s	remaining: 7m 14s
500:	learn: 0.7326769	test: 0.7216451	best: 0.7216451 (500)	total: 7m 16s	remaining: 5m 45s
600:	learn: 0.7382494	test: 0.7255608	best: 0.7255608 (600)	total: 8m 43s	remaining: 4m 18s
700:	learn: 0.7436296	test: 0.7287731	best: 0.7287731 (700)	total: 10m 9s	remaining: 2m 51s
800:	learn: 0.7486776	test: 0.7319208	best: 0.7320009 (798)	total: 11m 36s	remaining: 1m 24s
897:	learn: 0.7536420	test: 0.7355886	best: 0.7356521 (894)	total: 13m 1s	remaining: 0us

bestTest = 0.7356521041
bestIteration = 894

Shrink model to first 895 iterations.


[I 2025-07-14 17:23:27,580] Trial 14 finished with value: 0.90403125 and parameters: {'iterations': 898, 'learning_rate': 0.03952389587309868, 'depth': 10, 'l2_leaf_reg': 3.188652353245689, 'border_count': 244, 'random_strength': 0.5502698980015268, 'bagging_temperature': 0.5063266063933292}. Best is trial 12 with value: 0.9107479166666667.


0:	learn: 0.6613395	test: 0.6596480	best: 0.6596480 (0)	total: 928ms	remaining: 17m 59s
100:	learn: 0.7127977	test: 0.7076007	best: 0.7076007 (100)	total: 1m 26s	remaining: 15m 10s
200:	learn: 0.7249776	test: 0.7168420	best: 0.7168420 (200)	total: 2m 51s	remaining: 13m 43s
300:	learn: 0.7361241	test: 0.7234043	best: 0.7234043 (300)	total: 4m 17s	remaining: 12m 18s
400:	learn: 0.7464443	test: 0.7304606	best: 0.7305370 (398)	total: 5m 44s	remaining: 10m 55s
500:	learn: 0.7553442	test: 0.7357434	best: 0.7357434 (500)	total: 7m 9s	remaining: 9m 29s
600:	learn: 0.7641902	test: 0.7409710	best: 0.7409710 (600)	total: 8m 34s	remaining: 8m 3s
700:	learn: 0.7719327	test: 0.7458394	best: 0.7459165 (699)	total: 10m 1s	remaining: 6m 38s
800:	learn: 0.7796775	test: 0.7503805	best: 0.7503805 (800)	total: 11m 26s	remaining: 5m 12s
900:	learn: 0.7870346	test: 0.7548655	best: 0.7548655 (900)	total: 12m 52s	remaining: 3m 46s
1000:	learn: 0.7937903	test: 0.7588111	best: 0.7588111 (1000)	total: 14m 17s	rem

[I 2025-07-14 17:40:29,824] Trial 15 finished with value: 0.9141666666666667 and parameters: {'iterations': 1165, 'learning_rate': 0.07478361216767605, 'depth': 10, 'l2_leaf_reg': 4.086922676710861, 'border_count': 226, 'random_strength': 0.9749389319660008, 'bagging_temperature': 0.6217145182218249}. Best is trial 15 with value: 0.9141666666666667.


0:	learn: 0.6580936	test: 0.6562820	best: 0.6562820 (0)	total: 868ms	remaining: 18m 19s


In [ ]:
# 이렇게 하면 총 10번 추가로 돌아감
# study.optimize(objective, n_trials=10)

In [ ]:
# 결과 출력 (궁금하니까 --> 여기선 F1이 micro가 아니라 낮을 수 밖에 없다.)
print("Best F1 score:", study.best_value)
print("Best parameters:", study.best_params)

In [ ]:
# 최적 파라미터로 모델 학습
# Optuna가 튜닝한 것이 아닌 것들은 다시 넣어줘야함.
# Optuna에 넣은 이유는 기본적으로 들어있어야 하기 때문에
best_params = study.best_params
best_params.update({
    "loss_function": "Logloss",
    "eval_metric": "F1",
    "random_state": 42,
    "verbose": 100
})

In [ ]:
# 실제 모델
model = CatBoostClassifier(**best_params)
model.fit(X_train, y_train,
          eval_set=(X_val, y_val),
          early_stopping_rounds=50)

In [ ]:
# 예측
test_df['Segment_pred'] = model.predict(X_test)

In [ ]:
test_df['Segment_pred'].value_counts()

### 2단계 -> Segment == C or D vs Other (E 제거)

In [ ]:
train_df2= train_df[train_df['Segment'] != 'E'].copy()
train_df2['Segment1'] = train_df2['Segment'].apply(lambda x: x if x in ['C', 'D'] else 'other')

In [ ]:
exclude_cols = ['ID', '기준년월', 'Segment', 'Segment1']
feature_cols = [
    col for col in train_df.columns
    if col not in exclude_cols
]

In [ ]:
# 학습용 피처/타겟 정의
X2 = train_df2[feature_cols]
y2 = train_df2['Segment1']

In [ ]:
# 예측할 test subset (1단계 결과가 Other인 경우)
test_df2 = test_df[test_df['Segment_pred'] == 'other'].copy()

In [ ]:
# X와 test_df 양쪽에 모두 존재하는 컬럼만 선택
common_cols = [col for col in X2.columns if col in test_df2.columns]
X2 = X2[common_cols]
X_test2 = test_df2[common_cols]

In [ ]:
# 학습/검증 분리
X_train, X_val, y_train, y_val = train_test_split(X2, y2, test_size=0.2, random_state=42, stratify=y2)

In [ ]:
def objective2(trial):
    params = {
        # boosting 반복 횟수
        "iterations": trial.suggest_int("iterations", 300, 1000),
        # 한번에 얼마나 빠르게 학습하지지
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
        # 트리의 깊이, 깊을수록 복잡한 모델이지만 과적합 가능성 O
        "depth": trial.suggest_int("depth", 4, 8), # --> 튜닝
        # L2 정규화 , 높을수록 규제 강화 - 과적합 억제
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 3, 10),
        # 수치형 피쳐를 이산화 할 때 사용하는 경계 개수 -> 수치형 많으면 영향력 있음
        "border_count": trial.suggest_int("border_count", 32, 128),
        # 트리 분기시 f.i 계산에 노이즈를 얼마나 줄 것인가
        "random_strength": trial.suggest_float("random_strength", 0.5, 1.0),
        # 샘플링의 다양성 조절 -> 과소적합일 땐 높이고, 과적합일 땐 낮게
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.2, 1.0),
        # 손실 함수 : 다중분류
        "loss_function": "MultiClass",
        # 평가 지표
        "eval_metric": "TotalF1:average=Micro",
        # 학습 로그를 몇 step마다 보여줄지
        "verbose": 100,
        # 트리 분기 시 feature 중요도 계산에 주는 노이즈 크기
        "random_state": 42
    }
    # early_stopping_rounds - 지정된 라운드 동안 성능 개선이 되지 않으면 학습 중지
    model2 = CatBoostClassifier(**params)
    model2.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=30, verbose=100)

    preds = model2.predict(X_val)
    f1 = f1_score(y_val, preds, average='micro')
    return f1

In [ ]:
# Optuna 튜닝 시작
study2 = optuna.create_study(direction="maximize")
# XGB 때 n_trials=30 으로 했었는데 너무 오래걸려서 일단 20으로 설정
# n_trials는 optuna가 시도할 하이퍼파라미터 조합의 개수 (몇번 학습할건지)
study2.optimize(objective2, n_trials=20)

In [ ]:
# 결과 출력 (궁금하니까 --> 여기선 F1이 micro가 아니라 낮을 수 밖에 없다.)
print("Best F1 score:", study2.best_value)
print("Best parameters:", study2.best_params)

In [ ]:
# 최적 파라미터로 모델 학습
# Optuna가 튜닝한 것이 아닌 것들은 다시 넣어줘야함.
# Optuna에 넣은 이유는 기본적으로 들어있어야 하기 때문에
best_params2 = study2.best_params
best_params2.update({
    "loss_function": "MultiClass",
    "eval_metric": "TotalF1:average=Micro",
    "random_state": 42,
    "verbose": 100
})

In [ ]:
# 실제 모델
model2 = CatBoostClassifier(**best_params2)
model2.fit(X_train, y_train,
          eval_set=(X_val, y_val),
          early_stopping_rounds=30)

In [ ]:
# 예측
test_df2['Segment_pred'] = model2.predict(X_test2).flatten()

In [ ]:
test_df2['Segment_pred'].value_counts()

In [ ]:
# 예측 결과 병합
# 중복된 ID 제거: 마지막 값을 기준으로 유지
test_df2_dedup = test_df2.drop_duplicates(subset='ID', keep='last')
test_df2_indexed = test_df2_dedup.set_index('ID')

# test_df의 Segment_pred를 ID 기준으로 업데이트
test_df.loc[test_df['ID'].isin(test_df2_indexed.index), 'Segment_pred'] = \
    test_df.loc[test_df['ID'].isin(test_df2_indexed.index), 'ID'].map(test_df2_indexed['Segment_pred'])

### 3단계 -> Segment == a or B vs Other (C, D, E 제거)

In [ ]:
# 학습용 피처/타겟 정의
train_df3 = train_df[train_df['Segment'].isin(['A', 'B'])].copy()
train_df3['Segment1'] = train_df3['Segment']

In [ ]:
exclude_cols = ['ID', '기준년월', 'Segment', 'Segment1']
feature_cols = [
    col for col in train_df.columns
    if col not in exclude_cols
]

In [ ]:
# 학습용 피처/타겟 정의
X3 = train_df3[feature_cols]
y3 = train_df3['Segment1']

In [ ]:
# 예측할 test subset (1단계 결과가 Other인 경우)
test_df3 = test_df[test_df['Segment_pred'] == 'other'].copy()

In [ ]:
# X와 test_df 양쪽에 모두 존재하는 컬럼만 선택
common_cols = [col for col in X3.columns if col in test_df3.columns]
X3 = X3[common_cols]
X_test3 = test_df3[common_cols]

In [ ]:
# 학습/검증 분리
X_train, X_val, y_train, y_val = train_test_split(X3, y3, test_size=0.2, random_state=42, stratify=y3)

In [ ]:
def objective3(trial):
    params = {
        # boosting 반복 횟수
        "iterations": trial.suggest_int("iterations", 100, 500),
        # 한번에 얼마나 빠르게 학습하지지
        "learning_rate": trial.suggest_float("learning_rate", 0.02, 0.2),
        # 트리의 깊이, 깊을수록 복잡한 모델이지만 과적합 가능성 O
        "depth": trial.suggest_int("depth", 3, 6), # --> 튜닝
        # L2 정규화 , 높을수록 규제 강화 - 과적합 억제
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 5, 20),
        # 수치형 피쳐를 이산화 할 때 사용하는 경계 개수 -> 수치형 많으면 영향력 있음
        "border_count": trial.suggest_int("border_count", 32, 64),
        # 트리 분기시 f.i 계산에 노이즈를 얼마나 줄 것인가
        "random_strength": trial.suggest_float("random_strength", 0.8, 2.0),
        # 샘플링의 다양성 조절 -> 과소적합일 땐 높이고, 과적합일 땐 낮게
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.5, 1.0),
        # 손실 함수 : 다중분류
        "loss_function": "MultiClass",
        # 평가 지표
        "eval_metric": "TotalF1:average=Micro",
        # 학습 로그를 몇 step마다 보여줄지
        "verbose": 100,
        # 트리 분기 시 feature 중요도 계산에 주는 노이즈 크기
        "random_state": 42
    }
    # early_stopping_rounds - 지정된 라운드 동안 성능 개선이 되지 않으면 학습 중지
    model3 = CatBoostClassifier(**params)
    model3.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=20, verbose=100)

    preds = model3.predict(X_val)
    f1 = f1_score(y_val, preds, average='micro')
    return f1

In [ ]:
# Optuna 튜닝 시작
study3 = optuna.create_study(direction="maximize")
# XGB 때 n_trials=30 으로 했었는데 너무 오래걸려서 일단 20으로 설정
# n_trials는 optuna가 시도할 하이퍼파라미터 조합의 개수 (몇번 학습할건지)
study3.optimize(objective3, n_trials=15)

In [ ]:
# 결과 출력 (궁금하니까 --> 여기선 F1이 micro가 아니라 낮을 수 밖에 없다.)
print("Best F1 score:", study3.best_value)
print("Best parameters:", study3.best_params)

In [ ]:
# 최적 파라미터로 모델 학습
# Optuna가 튜닝한 것이 아닌 것들은 다시 넣어줘야함.
# Optuna에 넣은 이유는 기본적으로 들어있어야 하기 때문에
best_params3 = study3.best_params
best_params3.update({
    "loss_function": "MultiClass",
    "eval_metric": "TotalF1:average=Micro",
    "random_state": 42,
    "verbose": 100
})

In [ ]:
# 실제 모델
model3 = CatBoostClassifier(**best_params3)
model3.fit(X_train, y_train,
          eval_set=(X_val, y_val),
          early_stopping_rounds=20)

In [ ]:
# 예측
test_df3['Segment_pred'] = model3.predict(X_test3).flatten()

In [ ]:
test_df3['Segment_pred'].value_counts()

In [ ]:
# 예측 결과 병합
# 중복된 ID 제거: 마지막 값을 기준으로 유지
test_df3_dedup = test_df3.drop_duplicates(subset='ID', keep='last')
test_df3_indexed = test_df3_dedup.set_index('ID')

# test_df의 Segment_pred를 ID 기준으로 업데이트
test_df.loc[test_df['ID'].isin(test_df3_indexed.index), 'Segment_pred'] = \
    test_df.loc[test_df['ID'].isin(test_df3_indexed.index), 'ID'].map(test_df3_indexed['Segment_pred'])

# 최종 예측 결과 저장
submission = test_df[['ID', 'Segment_pred']].copy()
submission.rename(columns={'Segment_pred': 'Segment'}, inplace=True)
submission.to_parquet(f'../results/{train_file}_catboost_model3_submission5.parquet', index=False)
print(f"{train_file}_catboost_model3_submission 저장 완료!")